In [1]:
import os
import numpy as np
import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
n_epochs = 200
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 1000
img_size = 28
channels = 1
sample_interval = 500

img_shape = (channels, img_size, img_size)
cuda = True if torch.cuda.is_available() else False

In [3]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [4]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [5]:
transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            ])
imagenet_data = torchvision.datasets.ImageFolder(root = '/media/arg_ws3/5E703E3A703E18EB/data/test',\
                                                transform = transform)
dataloader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=batch_size,
                                          shuffle=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=batch_size, shuffle=True)
it = iter(dataloader)

In [6]:
item = it.next()
imgs = item[0]
item[0].shape, item[1]

(torch.Size([64, 1, 28, 28]),
 tensor([6, 8, 5, 7, 3, 9, 8, 1, 7, 7, 4, 5, 0, 6, 1, 9, 1, 9, 0, 5, 1, 7, 2, 7,
         0, 7, 5, 3, 9, 4, 1, 6, 9, 3, 9, 3, 8, 4, 3, 9, 4, 3, 6, 7, 3, 5, 4, 0,
         5, 7, 9, 7, 9, 5, 4, 6, 5, 5, 5, 1, 9, 2, 0, 6]))

In [7]:
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False) # ([batch_size, 1]) filled with 1.
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False) # ([batch_size, 1]) fiiled with 0.

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim)))) # ([batch_size, latent_dim])

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        #print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, n_epochs, i, len(dataloader),
        #                                                    d_loss.item(), g_loss.item()))

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            print(labels)
            save_image(gen_imgs.data[:25], 'images/%d.png' % batches_done, nrow=5, normalize=True)

tensor([4, 4, 8, 1, 9, 1, 6, 0, 0, 7, 0, 7, 8, 8, 3, 1, 4, 2, 3, 6, 0, 6, 2, 9,
        1, 7, 2, 4, 1, 3, 1, 2, 3, 5, 2, 1, 5, 7, 6, 2, 7, 4, 5, 5, 4, 9, 4, 6,
        7, 4, 1, 1, 7, 6, 1, 0, 6, 5, 2, 1, 9, 2, 8, 1])
tensor([5, 7, 9, 4, 7, 8, 2, 3, 1, 9, 0, 3, 2, 2, 4, 1, 9, 3, 2, 8, 5, 2, 6, 6,
        6, 2, 6, 3, 4, 5, 0, 3, 0, 2, 7, 5, 2, 6, 4, 1, 5, 7, 9, 1, 4, 2, 3, 8,
        2, 4, 3, 5, 7, 1, 2, 5, 1, 6, 2, 3, 8, 9, 0, 8])
tensor([9, 1, 2, 6, 6, 8, 8, 3, 3, 1, 8, 5, 0, 3, 0, 5, 7, 9, 1, 0, 3, 7, 5, 8,
        4, 5, 1, 9, 9, 6, 6, 9, 7, 7, 4, 3, 3, 4, 3, 5, 5, 0, 4, 9, 9, 3, 5, 9,
        4, 5, 6, 9, 4, 1, 9, 8, 9, 5, 8, 2, 8, 6, 1, 6])
tensor([5, 7, 5, 9, 0, 4, 5, 1, 9, 1, 8, 9, 6, 4, 1, 0, 6, 1, 3, 2, 7, 7, 0, 8,
        7, 6, 0, 9, 1, 2, 7, 9, 7, 1, 2, 2, 5, 7, 6, 5, 0, 3, 8, 2, 9, 5, 1, 8,
        8, 1, 6, 4, 5, 0, 1, 7, 9, 9, 3, 6, 9, 7, 6, 7])
tensor([6, 3, 8, 6, 6, 9, 4, 7, 1, 2, 4, 4, 5, 1, 7, 2, 7, 2, 3, 2, 4, 8, 7, 3,
        4, 1, 1, 7, 3, 6, 4, 2, 1, 7, 5, 5, 8, 1, 1,

KeyboardInterrupt: 